In [1]:
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.geography_utils import CA_NAD83Albers_m, WGS84
gcsgp = GCSGeoPandas()
from shared_utils import gtfs_utils_v2
from shared_utils.rt_utils import show_full_df

import utils
import update_vars

# Create utils to get stop sequence and distance info for ridership charts

* currently blocked o

In [2]:
analysis_date = update_vars.ANALYSIS_DATE

In [3]:
ca = gcsgp.read_parquet('gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/ca_boundary.parquet')

## New SJJPA San Joaquins feed

In [4]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

In [5]:
sanj = feeds.query('name.str.contains("Amtrak San J")')
sanj

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,type,regional_feed_type
72,d531c079e5f22d71e999525d72e81453,2025-09-01,42ca573996762b5bf9b2930e0a0c9d8d,America/Los_Angeles,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,524ea6209600e9a2de34a02cf9068729,Amtrak San Joaquins Schedule,schedule,None


In [6]:
sanj_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=sanj.feed_key, shape_cols=update_vars.shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [7]:
sanj_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=sanj.feed_key, trip_cols=update_vars.trip_cols)

In [14]:
sanj_trips

,trip_instance_key,trip_id,trip_short_name,base64_url,feed_key,name,regional_feed_type,gtfs_dataset_key,service_date,direction_id,...,route_key,route_id,route_type,route_short_name,route_long_name,route_desc,agency_id,network_id,shape_array_key,shape_id


In [8]:
sanj_stops = gtfs_utils_v2.get_stops(analysis_date, operator_feeds=sanj.feed_key, stop_cols=update_vars.stop_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [9]:
sanj_st = gtfs_utils_v2.get_stop_times(analysis_date, operator_feeds=sanj.feed_key, get_df=True,
                                      trip_df = sanj_trips, stop_time_cols=update_vars.stop_time_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


In [10]:
sanj_trips.query('route_type == "2"')

,trip_instance_key,trip_id,trip_short_name,base64_url,feed_key,name,regional_feed_type,gtfs_dataset_key,service_date,direction_id,...,route_key,route_id,route_type,route_short_name,route_long_name,route_desc,agency_id,network_id,shape_array_key,shape_id


In [11]:
sanj_trips[['route_short_name', 'route_long_name', 'trip_instance_key']].groupby(['route_short_name', 'route_long_name']).count()

,,trip_instance_key
route_short_name,route_long_name,


In [12]:
sanj_shapes_trip_info = sanj_shapes.merge(sanj_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])

In [13]:
# sanj_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

In [41]:
sanj_trips.query('route_short_name == "Route 99"')

,trip_instance_key,trip_id,trip_short_name,base64_url,feed_key,name,regional_feed_type,gtfs_dataset_key,service_date,direction_id,...,route_key,route_id,route_type,route_short_name,route_long_name,route_desc,agency_id,network_id,shape_array_key,shape_id
